AI Based Currency Converter

In [2]:
pip install -q langchain-core langchain-groq requests python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 3.9 MB/s eta 0:00:00


In [3]:
import os
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

Building Tool

In [4]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> str:
    """This tool returns the conversion factor from base currency to target currency"""

    # Get the API key from Colab secrets
    from google.colab import userdata
    api_key = userdata.get("EXCHANGE_RATE_API_KEY")

    url = f'https://v6.exchangerate-api.com/v6/{api_key}/latest/{base_currency}'

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        data = response.json()
        # Check if the target currency exists in the conversion rates
        if target_currency in data.get('conversion_rates', {}):
            return data['conversion_rates'][target_currency]
        else:
            print(f"Error: Target currency '{target_currency}' not found in conversion rates.")
            print(f"API Response: {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching conversion rate: {e}")
        print(f"API Response (if available): {response.text if 'response' in locals() else 'N/A'}")
        return None
    except ValueError as e:
        print(f"Error decoding JSON response: {e}")
        print(f"API Response: {response.text}")
        return None

@tool
def convert(base_currency_valu: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """This tool converts the base currency value to target currency"""
    return base_currency_valu * conversion_rate

Tool Binding

In [5]:
from google.colab import userdata

llm = ChatGroq(
    api_key=userdata.get("GROQ_API_KEY"),
    model="llama-3.1-8b-instant"
)

In [6]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [7]:
messages = [HumanMessage("What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr")]

In [90]:
ai_message = llm_with_tools.invoke(messages)

In [91]:
messages.append(ai_message)

In [92]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'rednfdmdn',
  'type': 'tool_call'},
 {'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'sj31cb9by',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency': 'USD',
   'base_currency_valu': 10,
   'target_currency': 'INR'},
  'id': 'ad8n8g0nz',
  'type': 'tool_call'}]

In [101]:
import json
from langchain_core.messages import ToolMessage

for tool_call in ai_message.tool_calls:

    # execute the 1st tool and get the value of conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)  # already returns float
        conversion_rate = tool_message1.content  # use float directly
        messages.append(tool_message1)

    # execute the 2nd tool using the conversion rate from tool 1
    if tool_call['name'] == 'convert':
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)

In [102]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'rednfdmdn', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'sj31cb9by', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'ad8n8g0nz', 'function': {'arguments': '{"base_currency":"USD","base_currency_valu":10,"target_currency":"INR"}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 326, 'total_tokens': 407, 'completion_time': 0.093072801, 'prompt_time': 0.076347381, 'queue_time': 1.130860113, 'total_time': 0.169420182}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_55d70a61e4', 'service_ti

In [104]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR is 88.2133. \nBased on that, 10 USD can be converted to 882.133 INR.'